In [1]:
from numpy.testing import assert_, assert_equal, assert_almost_equal, assert_allclose, assert_approx_equal, assert_raises, assert_allclose
import numpy as np #import numpy as usual           

##### ME 574 Spring 2021

# Homework 3

1. Fill in python code to implement the function `run_avg(a, rad)` that computes the running average over `(2*r+1)` consecutive entries from the array `a`.

In [2]:
def run_avg(a, r):
    """
    Compute the running average over (2*r+1) consecutive elements of an input array.
    The rad entries at the beginning and end of the output array should agree with those of the input array.
    
    Arguments:
        a: float numpy array of input data
        r: int averaging radius
        
    Returns:
        avg: numpy float array of values over 2*r+1 successive entries in a.
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    # Based on CH5_Stencils_Shared_Mem Slides
    n = a.shape[0]
    avg = np.zeros(n)

    for i in range(n):
      if i < r:
        avg[i] = a[i]
      elif i >= n-r:
        avg[i] = a[i]
      else:
        tmp = 0
        
        for j in range(2*r + 1):
            ind = i + j - r 
            tmp = tmp + a[ind]

        avg[i] = tmp/(2.*r + 1.)
    return avg

In [3]:
len = 6
nil_array = np.zeros(len)
one_array = np.ones(len)

test_data = np.concatenate((nil_array, one_array))
desired2 = np.array([0. , 0. , 0. , 0. , 0.2, 0.4, 0.6, 0.8, 1. , 1. , 1. , 1. ])
assert_allclose(run_avg(test_data, 2), desired2)
desired3 = np.array([0.        , 0.        , 0.        , 0.14285714, 0.28571429,
       0.42857143, 0.57142857, 0.71428571, 0.85714286, 1.        ,
       1.        , 1.        ])
assert_allclose(run_avg(test_data, 3), desired3)

2. Write python code to implement a parallel version `p_run_avg(a,r)`. Use __global memory__ for this version.

In [4]:
from numba import cuda
from numba import float32
TPB = 512

@cuda.jit
def avg_kernel(d_out, d_a, r):
    """
    Kernel function for parallel computation of running average.
    
    Arguments:
        d_out: float device array for storing output
        d_a: float device copy of input data array
        r: int radius
        
    Returns:
        None
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    # Based on CH5_Stencils_Shared_Mem Slides

    i = cuda.grid(1)
    n = d_a.shape[0]
    if i < n:
      if i < r:
        d_out[i] = d_a[i]
      elif(i >= n-r):
        d_out[i] = d_a[i]
      else:
        tmp = 0
        for j in range(2*r + 1):
          ind = i + j - r 
          tmp = tmp + d_a[ind]
        d_out[i] = tmp/(2*r + 1)

def p_run_avg(a, r):
    """
    Compute the running average over (2*r+1) consecutive elements of an input array in parallel using global memory.
    The rad entries at the beginning and end of the output array should agree with those of the input array.
    
    Arguments:
        a: float numpy array of input data
        r: int averaging radius
        
    Returns:
        avg: numpy float array of values over 2*r+1 successive entries in a.
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    n = a.shape[0] 
    d_a = cuda.to_device(a)
    d_out = cuda.device_array_like(d_a)
    blockDim = TPB 
    gridDim = (n + TPB - 1)//TPB
    avg_kernel[gridDim,blockDim](d_out, d_a, r)
    avg = d_out.copy_to_host()

    return avg

In [5]:
TPB = 32
N =  1<<6

nil_array = np.zeros(N)
one_array = np.ones(N)
test_data = np.concatenate((nil_array, one_array))

assert_allclose(p_run_avg(test_data, 2), run_avg(test_data, 2))
assert_allclose(p_run_avg(test_data, 3), run_avg(test_data, 3))

3. Write python code to implement a parallel version `s_run_avg(a,r)`. Use a __shared memory array__ for this version.

In [6]:
from numba import jit,cuda,float32
#TPB, RAD = 128, 1
#NSHARED=130 #mustagreewithTPB+2*RAD


@cuda.jit
def s_avg_kernel(d_out, d_a, r):
    """
    Kernel function for parallel computation of running average using shared memory array.
    
    Arguments:
        d_out: float device array for storing output
        d_a: float device copy of input data array
        r: int radius
        
    Returns:
        None
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    # Based on Stencils with shared Arrays PDF

    n = d_a.shape[0]
    i = cuda.grid(1)
    sh_a = cuda.shared.array(NSHARED, dtype = np.float32)
    tIdx = cuda.threadIdx.x #thread index
    if tIdx.x == 0:
        print(cuda.blockIdx.x)
    shIdx = tIdx + rad #shared index
    if i>=n:
        return #bounds check
    sh_a[shIdx] = d_a[i] #load regular cells

    if tIdx < rad: #Halo cells: check bounds
        if i >= rad:
            sh_a[shIdx - rad] = d_a[i-rad]
        if i + cuda.blockDim.x < n:
            sh_a[shIdx + cuda.blockDim.x] = d_a[i + cuda.blockDim.x]

    cuda.syncthreads() #sync before read
    
    i = cuda.grid(1)
    n = d_a.shape[0]
    if i < n:
      if i < r:
        d_out[i] = d_a[i]
      elif(i >= n-r):
        d_out[i] = d_a[i]
      else:
        tmp = 0
        for j in range(2*r + 1):
          ind = i + j - r 
          tmp = tmp + d_a[ind]
        d_out[i] = tmp/(2*r + 1)

def s_run_avg(a, r):
    """
    Compute the running average over (2*r+1) consecutive elements of an input array in parallel using shared memory.
    The rad entries at the beginning and end of the output array should agree with those of the input array.
    
    Arguments:
        a: float numpy array of input data
        r: int averaging radius
        
    Returns:
        avg: numpy float array of values over 2*r+1 successive entries in a.
    """
    # YOUR CODE HERE
    # raise NotImplementedError()

    n = a.shape[0] 
    d_a = cuda.to_device(a)
    d_out = cuda.device_array_like(d_a)
    blockDim = TPB 
    gridDim = (n + TPB - 1)//TPB
    avg_kernel[gridDim,blockDim](d_out, d_a, r)
    avg = d_out.copy_to_host()

    return avg

In [7]:
TPB = 32
N =  1<<6

nil_array = np.zeros(N)
one_array = np.ones(N)
test_data = np.concatenate((nil_array, one_array))

assert_allclose(s_run_avg(test_data, 2), run_avg(test_data, 2))
assert_allclose(s_run_avg(test_data, 3), run_avg(test_data, 3))

The remaining problems focus on Laplace's equation, $\nabla^2 u = 0$, which arises frequently is studies of steady-state diffusion (including diffusion of heat as a system evolves toward a sgteady-state temperature distrbution).

In 1D Cartesian coordinates, Laplace's equation simplifies to $\frac{d^2 u}{dx^2} = 0$. A common approach to computing  approximate solutions involves discretizing the domain to a grid of $N+1$ regularly spaced points $x_i = i x + x_o$ and applying a central difference approximation for the derivative to obtain $x_{i-1} -2 x_i + x_{i+1} = 0, i \in [1, \ldots, N]$.

There are 2 distinct ways to proceed toward a solution from this underlying difference equation: treat the collection of $N-1$ equations together as a big linear algebra problem to be solved for the vector of steady-state values on the collection of grid points or use the difference equation as the basis for an iteration scheme to update values at individual grid points. In this problem, we pursue the latter approach and construct an iteration scheme (known as __Jacobi iteration__) as follows:

- Solve the difference equation for $x_i = \frac{1}{2} (x_{i-1} + x_{i+1})$
- Include a superscript to designate the iteration number and update the value of $x_i$ based on the values at the neighboring grid points from the previous iteration: $x_i^{k+1} = \frac{1}{2} (x_{i-1}^k + x_{i+1}^k)$.
- From an initial guess of the solution, update the value at each point (i.e. replace the value with the average of the values at the neighboring grid points) and repeat until the values no longer change significantly during an update step.

 
4. Use the `@cuda.reduce` decorator to write a function `norm_diff(u,v)` that measures the difference between arrays. It will be used to test if the "solution" changes significantly during an update step. Your implementation of `normdiff(u,v)` should implement the $L^1$-norm: $|u-v|_1 = \sum{| u_i - v_i |}$.

In [8]:
# implement the reduction operator

# write a reduction `sum_abs(a,b)` operator to sum absolute values
# YOUR CODE HERE
# raise NotImplementedError()

@cuda.reduce
def sum_abs(a,b):
    return abs(a) + abs(b)

#implement the L1 norm using the `sum_abs()` reduction
def norm_diff(u,v):
    """
    Compute the L^1 norm of the difference between arrays
    
    Arguments:
        u, v: numpy arrays
        
    Returns:
        value: float norm equal to sum of absolute values of difference between corr. entries
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    # Based on Difference between two numpy arrays in python/StackOverflow
    
    value = sum_abs(np.subtract(u,v))
    return value

In [9]:
N = 100
v = np.ones(N)
assert_(sum_abs(v) == np.abs(v).sum())
v = np.array([(-1)**i*(i+1) for i in range(100)])
assert_(sum_abs(v) == np.abs(v).sum())

Below is code for a serial implementation of the function `jacobi_update(u)` that returns $u^{k+1}$ given $u^k$. 
This implementation creates a copy of the input array, writes the updated values into the copy, and returns the updated version of the copy.

In [10]:
def jacobi_update(u):
    """
    Perform one Jacobi iteration step:
    
    Arguments:
        u: numpy array of current values
        
    Returns:
        u_new: float numpy array of updated values (corr. to avg. of neighboring input values)
    """
    # YOUR CODE HERE
    # raise NotImplementedError()

    n = u.shape[0]
    u_new = np.copy(u)
    for i in range(1 , n - 1):
        u_new[i] = (u[i - 1]+u[i +1 ])/2.
    return u_new

In [11]:
# test to check that jacobi-update works as expected
N = 8
u = np.zeros(N)
u[-1] = 1.
assert_allclose(jacobi_update(u), np.array([0. , 0., 0. , 0. , 0. , 0. , 0.5, 1. ]))

The cell below shows a serial implementation of the function `jacobi_solve(u, tol, max_iters)` that computes an approximate solution of the 1D Laplace equation by performing repeated Jacobi updates until reaching one of the following termination criteria:

- The number of updates reaches a specified maximum value, `max_iters`. (In this case, it prints a messaage warning of failed convergence.)
- The norm of the change in the array caused by an update is less than a specified value, `tol`.

In [12]:
def jacobi_solve(u, tol, max_iters):
    """
    Compute an approximate solution of 1D Laplace equation by repeated Jacobi updates.
    
    Arguments:
        u: numpy array of initial guess of solution values
        tol: float L1 norm of change in u at which to stop iterating
        max_iters: maximum number of updates to compute
        
    Returns:
        count: int number of iterations
        sol: numpy float array of approximate solution values
    """
    u_old =  np.copy(u)
    for count in range(1, max_iters+1):
        u_new = jacobi_update(u_old)
        if norm_diff(u_new, u_old)<tol:
            sol = u_new
            return count, sol
        u_old = np.copy(u_new)
        sol = u_new
    return count, sol

In [13]:
# test jacobi_solve to see if it works as expected
N = 11
u = np.zeros(N)
u[-1] = 1.
tol = 0.00005
assert_allclose(jacobi_solve(u, tol, 1)[1], np.array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0., 0.5 , 1.  ]))
assert_allclose(jacobi_solve(u, tol, 200)[1], 0.1 * np.arange(11), rtol = 0, atol=1e-3)

In [14]:
# test jacobi_solve to see if it works as expected
N = 11
u = np.zeros(N)
u[-1] = 1.
tol = 0.00005
assert_allclose(jacobi_solve(u, tol, 1)[1], np.array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0., 0.5 , 1.  ]))
assert_allclose(jacobi_solve(u, tol, 200)[1], 0.1 * np.arange(11), rtol = 0, atol=1e-3)

5. Code up a parallel implementation of `jacobi_solve()` using __global memory__. To retain the current values (for both later reads and comparison to updated values), use separate arrays for input data and output data. 

Recall that these will need to be device arrays, and there are limits to the memory available on the device. The recommended strategy is to create 2 arrays of the necessary size in global memory and, on subsequent update steps, alternate which array is used for input/output.

Also, remember that copying data back and forth between the host and device would incur a performance penalty, so keep the data on the device side, and only copy the array of final values back to the host side for inspection.

In [15]:
# insert your global memory parallel implementation of jacobi_solve below

TPB = 32

# @cuda.jit(device=True)
def norm_diff(u, v):
    """
    Compute the L^1 norm of an array
    
    Arguments:
        u: numpy array
        
    Returns:
        value: float norm equal to sum of absolute values of entries
    """
    # YOUR CODE HERE
    # raise NotImplementedError()
    # Based on Difference between two numpy arrays in python/StackOverflow
    value = sum_abs(np.subtract(u,v))
    return value

@cuda.jit
def update_kernel(d_v, d_u):
    """
    Kernel for parallel jacobi update using global memory.
    """
    # YOUR CODE HERE
    # raise NotImplementedError()

    n = d_u.shape[0]
    i = cuda.grid(1)

    if i >= n:
        return
    if i > 0 and i < n - 1:
        d_v[i] = 0.5*(d_u[i-1]+d_u[i+1])
    
def p_jacobi_update(d_v, d_u):
    """
    Perform one Jacobi iteration step in parallel using global memory.
    
    Arguments:
        d_u: numpy array of current values
        d_v: numpy array for sotring updated values
        
    Returns:
        None
    """
    # YOUR CODE HERE
    #raise NotImplementedError()

    n = d_u.shape[0]
    blocks = (n+TPB-1)//TPB
    update_kernel[blocks, TPB](d_v, d_u)

def jacobi_solve(u, tol, max_iters):
    """
    Compute an approximate solution of 1D Laplace equation by repeated Jacobi updates.
    This implementation is parallelized using global memory.
    
    Arguments:
        u: numpy array of initial guess of solution values
        tol: float L1 norm of change in u at which to stop iterating
        max_iters: maximum number of updates to compute
        
    Returns:
        count: int number of update iterations
        sol: numpy float array of approximate solution values
    """
    # YOUR CODE HERE
    # raise NotImplementedError()

    d_u =  cuda.to_device(u)
    d_v = cuda.to_device(u)

    for k in range(1, max_iters+1, 2):
        p_jacobi_update(d_v, d_u)
        if norm_diff(d_v, d_u)<tol:
            sol = d_v.copy_to_host()
            count = k
            break

        p_jacobi_update(d_u, d_v)
        if norm_diff(d_v, d_u)<tol:
            sol = d_u.copy_to_host
            count = k+1
            break

    sol = d_u.copy_to_host()
    count = k + 1

    return count, sol

In [16]:
N = 11
u = np.zeros(N)
u[-1] = 1.
tol = 0.00005
count, sol = jacobi_solve(u, tol, 2)
assert_(count==2)
assert_allclose(sol, np.array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.5 , 1.  ]))
count, sol = jacobi_solve(u, tol, 200)
assert_(count == 166)
assert_allclose(sol, 0.1 * np.arange(11), rtol = 0, atol=1e-3)

6. Write code to implement a parallel implementation of `jacobi_solve()` using __shared memory__.

In [17]:
# insert your shared memory parallel implementation of jacobi_solve below

TPB = 32
RAD = 1
N_SH = TPB + 2*RAD


@cuda.jit
def update_kernel(d_v, d_u):
    """
    Kernel for parallel jacobi update using shared memory.
    """
    # YOUR CODE HERE
    # raise NotImplementedError()

    n = d_u.shape[0]
    i = cuda.grid(1)
    threads = cuda.blockDim.x
    tIdx = cuda.threadIdx.x
    sh_i = tIdx + RAD
    sh_u = cuda.shared.array(N_SH,dtype = np.float32)
    sh_u[N_SH] = d_u[i]

    if tIdx < RAD:
      if i>= RAD:
        sh_u[sh_i - RAD] = d_u[i - RAD]
      if i + threads > n:
        sh_u[sh_i + threads] = d_u[i + threads]
    cuda.syncthreads()

    if i > 0 and i < n - 1:
      d_v[i] = 0.5* (d_u[i-1] + d_u[i+1])  
    
def p_jacobi_update(d_v, d_u):
    """
    Perform one Jacobi iteration step in parallel using global memory.
    
    Arguments:
        d_u: numpy array of current values
        d_v: numpy array for sotring updated values
        
    Returns:
        None
    """
    # YOUR CODE HERE
    # raise NotImplementedError()

    n = d_u.shape[0]
    blocks = (n+TPB-1)//TPB
    update_kernel[blocks, TPB](d_v, d_u)

def jacobi_solve(u, tol, max_iters):
    """
    Compute an approximate solution of 1D Laplace equation by repeated Jacobi updates.
    This implementation is parallelized using shared memory.
    
    Arguments:
        u: numpy array of initial guess of solution values
        tol: float L1 norm of change in u at which to stop iterating
        max_iters: maximum number of updates to compute
        
    Returns:
        count: int number of update iterations
        sol: numpy float array of approximate solution values
    """
    # YOUR CODE HERE
    # raise NotImplementedError()

    d_u =  cuda.to_device(u)
    d_v = cuda.to_device(u)

    for k in range(1, max_iters+1, 2):
        p_jacobi_update(d_v, d_u)
        if norm_diff(d_v, d_u)<tol:
            sol = d_v.copy_to_host()
            count = k
            break

        p_jacobi_update(d_u, d_v)
        if norm_diff(d_v, d_u)<tol:
            count = k+1
            sol = d_u.copy_to_host()
            break
            
        count = k+1
        sol = d_u.copy_to_host()
    return count, sol

In [18]:
N = 11
u = np.zeros(N)
u[-1] = 1.
tol = 0.00005
count, sol = jacobi_solve(u, tol, 2)
assert_(count == 2)
assert_allclose(sol, np.array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.5 , 1.  ]))
count, sol = jacobi_solve(u, tol, 200)
assert_(count == 166)
assert_allclose(sol, np.linspace(0., 1., 11), rtol = 0, atol=1e-3)